Imports

In [12]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset
from os import chdir, remove, close, listdir, path
from pydub import AudioSegment 


In [14]:
chdir("/home/rodoggx/iResearch-2024/ATC-recordings")
for foo in listdir():
    input_file = foo
    output_file = foo.rsplit('.', 1)[0] + '.wav'

    # convert mp3 file to wav file 
    sound = AudioSegment.from_mp3(input_file) 
    sound.export(output_file, format="wav")
    
    remove(foo)

In [16]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    torch_dtype=torch_dtype,
    device=device,
)

dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample = dataset[0]["audio"]

for foo in listdir():
    result = pipe(foo, return_timestamps=True)
    print(result)
    print("\n----------------------------------------------------------\n")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/rodoggx/iResearch-2024/venv/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:480: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
